In [1]:
import os

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GITHUB_TOKEN")
secret_value_1 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_2 = user_secrets.get_secret("HuggingFACEHUB_access_token")
secret_value_3 = user_secrets.get_secret("LANGCHAIN_API_KEY")

# ✅ IMPORTANT: Set them in os.environ so other code can access them
os.environ["GITHUB_TOKEN"] = secret_value_0
os.environ["GROQ_API_KEY"] = secret_value_1
os.environ["HuggingFACEHUB_access_token"] = secret_value_2
os.environ["LANGCHAIN_API_KEY"] = secret_value_3
os.environ["LLM_BACKEND"] = "vllm"

# ✅ NEW: Set vLLM model path (must match hirachical-subchat.ipynb)
model_path = "/kaggle/input/qwen-3/transformers/14b-awq/1"
os.environ["VLLM_MODEL_PATH"] = model_path

# Print the tokens (first 4 and last 4 characters for security)
print("="*60)
print("🔐 SECRETS LOADED AND SET IN ENVIRONMENT")
print("="*60)
print(f"✅ GITHUB_TOKEN: {secret_value_0[:4]}...{secret_value_0[-4:]}")
print(f"✅ GROQ_API_KEY: {secret_value_1[:4]}...{secret_value_1[-4:]}")
print(f"✅ HuggingFACEHUB_access_token: {secret_value_2[:4]}...{secret_value_2[-4:]}")
print(f"✅ LANGCHAIN_API_KEY: {secret_value_3[:4]}...{secret_value_3[-4:]}")
print(f"✅ LLM_BACKEND: vllm")
print(f"✅ VLLM_MODEL_PATH: {model_path}")
print("="*60)

🔐 SECRETS LOADED AND SET IN ENVIRONMENT
✅ GITHUB_TOKEN: gith...tWfg
✅ GROQ_API_KEY: gsk_...l6gr
✅ HuggingFACEHUB_access_token: hf_E...GaQC
✅ LANGCHAIN_API_KEY: lsv2...ea2f
✅ LLM_BACKEND: vllm
✅ VLLM_MODEL_PATH: /kaggle/input/qwen-3/transformers/14b-awq/1


# ⚙️ Configure Git for Auto-Push

**This cell configures git identity so the test runner can push results to GitHub after each buffer test completes.**

The `kaggle_buffer_test_runner.py` will:
- Run tests for buffer sizes: 5, 10, 20, 40
- **Push results to `kaggle-run` branch after EACH buffer completes**
- Stop execution if any push fails (ensures data integrity)
- Clear ChromaDB state between tests for clean measurements

In [3]:
# Configure git identity for Kaggle git push
import subprocess

REPO_DIR = "/kaggle/working/Subchat-Trees"

print("="*60)
print("⚙️  CONFIGURING GIT FOR KAGGLE")
print("="*60)

try:
    import os
    os.chdir(REPO_DIR)
    
    # Set git identity
    subprocess.run(["git", "config", "user.name", "moonmehedi"], check=True)
    subprocess.run(["git", "config", "user.email", "the.mehedi.hasan.moon@gmail.com"], check=True)
    
    print("✅ Git identity configured!")
    print(f"   User: moonmehedi")
    print(f"   Email: the.mehedi.hasan.moon@gmail.com")
    
    # Verify current branch
    branch_result = subprocess.run(["git", "branch", "--show-current"], capture_output=True, text=True)
    print(f"\n✅ Current branch: {branch_result.stdout.strip()}")
    
    # Check if GITHUB_TOKEN is available
    if "GITHUB_TOKEN" in os.environ:
        token_preview = os.environ["GITHUB_TOKEN"]
        print(f"\n✅ GITHUB_TOKEN available: {token_preview[:4]}...{token_preview[-4:]}")
    else:
        print("\n⚠️  WARNING: GITHUB_TOKEN not set - git push will fail!")
    
    os.chdir("/kaggle/working")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("="*60)

⚙️  CONFIGURING GIT FOR KAGGLE
✅ Git identity configured!
   User: moonmehedi
   Email: the.mehedi.hasan.moon@gmail.com

✅ Current branch: kaggle-run

✅ GITHUB_TOKEN available: gith...tWfg


# 🚀 Run Buffer Tests with Auto-Push

**Using `kaggle_buffer_test_runner.py` - Kaggle-aware version with automatic GitHub sync**

## What This Does:
1. Tests buffer sizes: **5, 10, 20, 40**
2. For each buffer:
   - Runs baseline test (single conversation, all topics mixed)
   - **Clears ChromaDB** (ensures clean state)
   - Runs system test (subchat trees, isolated contexts)
   - **Clears ChromaDB** again
   - Generates metrics tables
   - **Pushes results to GitHub kaggle-run branch**
   - **STOPS if push fails** (data integrity guarantee)
3. Generates final comparison visualization

## Expected Duration:
- ~10-15 minutes per buffer size
- **Total: ~40-60 minutes for all 4 buffer sizes**

## What Gets Pushed to GitHub After Each Buffer:
- `TABLE_1_CONTEXT_ISOLATION.md` (precision, recall, F1, accuracy)
- `TABLE_3_SYSTEM_PERFORMANCE.md` (tokens, latency, costs)
- `raw_metrics.json` (full test data)
- Log files (baseline_test.log, system_test.log)

You can pull these results locally to monitor progress in real-time!

In [4]:
# ✅ RUN KAGGLE BUFFER TEST RUNNER (with automatic git push after each buffer)
! python /kaggle/working/Subchat-Trees/backend/dataset/kaggle_buffer_test_runner.py

[14:36:07] [INFO] ================================================================================
[14:36:07] [INFO] 🚀 STARTING KAGGLE MULTI-BUFFER COMPARISON
[14:36:07] [INFO]    Buffer sizes: [5, 10, 20, 40]
[14:36:07] [INFO]    Git branch: kaggle-run
[14:36:07] [INFO] ================================================================================
[14:36:07] [INFO] 
[14:36:07] [INFO] 📦 TESTING BUFFER SIZE: 5
[14:36:07] [INFO] ================================================================================
[14:36:07] [INFO] ================================================================================
[14:36:07] [INFO] 🚀 STARTING METRICS-BASED EVALUATION (buffer_size=5)
[14:36:07] [INFO] ================================================================================
[14:36:07] [INFO] ⏳ Checking if server is ready...
[14:36:07] [INFO] ✅ Server is ready!
[14:36:07] [INFO] 
🔵 BASELINE TEST: 6c4992f0aed04dd3bf9a4bc225bb4fb0_structured.json (buffer_size=5)
[14:36:07] [INFO] 🗑️  Ensurin

# 🛑 Cleanup: Kill Server and Kernel

**This cell automatically shuts down the server and kernel to save Kaggle GPU quota.**

⚠️ **Run this ONLY after all tests are complete and results are pushed to GitHub!**

This cell will:
1. Kill the backend server process (uvicorn)
2. Terminate this notebook kernel
3. Free up GPU resources immediately

The notebook will stop executing after this runs.

In [ ]:
import subprocess
import os
import time

print("="*60)
print("🛑 SHUTTING DOWN SERVER AND KERNEL")
print("="*60)

# 1. Kill the backend server
try:
    print("\n🔴 Stopping backend server...")
    result = subprocess.run(
        ["pkill", "-f", "uvicorn.*src.main:app"],
        capture_output=True,
        text=True
    )
    time.sleep(2)
    print("✅ Backend server stopped")
except Exception as e:
    print(f"⚠️  Error stopping server: {e}")

# 2. Kill the kernel
print("\n🔴 Terminating kernel...")
print("✅ Kernel will shut down now - this saves GPU quota!")
print("="*60)

# Force exit the kernel
os._exit(0)

🛑 SHUTTING DOWN SERVER AND KERNEL

🔴 Stopping backend server...


: 

: 